In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score


import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
%cd "content/drive/My Drive/Colab Notebooks"

[Errno 2] No such file or directory: 'content/drive/My Drive/Colab Notebooks'
/content


In [7]:
ls

drive/  sample_data/


In [8]:
cd "drive"

/content/drive


In [9]:
ls

'My Drive'/


In [10]:
cd "My Drive"

/content/drive/My Drive


In [12]:
cd "Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [13]:
ls data


men_shoes.csv  men_shoes_f.csv  shoes_prices.csv


In [0]:
df = pd.read_csv("data/men_shoes_f.csv", low_memory=False)

In [17]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):

  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = "neg_mean_absolute_error")
  return(np.mean(scores), np.std(scores))

  

In [19]:
df["brand_cat"] = df["brand"].factorize()[0]
run_model(["brand_cat"])

(-58.38655694633361, 4.223555478221712)

In [20]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(["brand_cat"], model)

(-57.47223572384038, 4.328288468270897)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == "nan": return(output_dict)
  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item["key"].lower().strip()
    value = item["value"][0].lower().strip()

    output_dict[key] = value

  return(output_dict)

df["features_parsed"] = df["features"].map(parse_features)

In [32]:
df["features_parsed"].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [35]:
keys = set()

df["features_parsed"].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [39]:
def get_name_feat(key):
  return("feat_" + key)


for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stats = {}
for key in keys:
  keys_stats[key] = df[False == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0] *100

In [48]:
{k:v for k,v in keys_stats.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df["feat_brand_cat"] = df["feat_brand"].factorize()[0]
df["feat_color_cat"] = df["feat_color"].factorize()[0]
df["feat_gender_cat"] = df["feat_gender"].factorize()[0]
df["feat_material_cat"] = df["feat_material"].factorize()[0]
df["feat_manufacturer part number"] = df["feat_manufacturer part number"].factorize()[0]
df["feat_sport_cat"] = df["feat_sport"].factorize()[0]
df["feat_style_cat"] = df["feat_style"].factorize()[0]
df["feat_age_cat"] = df["feat_age"].factorize()[0]
df["feat_shape_cat"] = df["feat_shape"].factorize()[0]
df["feat_weight_cat"] = df["feat_weight"].factorize()[0]
df["feat_metal type_cat"] = df["feat_metal type"].factorize()[0]



In [57]:
df["brand"] = df["brand"].map(lambda x: str(x).lower())

df[df.brand == df.feat_brand].shape[0]

8846

In [74]:
feats = ["brand_cat", "feat_brand_cat", "feat_gender_cat", "feat_material_cat", "feat_style_cat", "feat_age_cat", "feat_shape_cat", "feat_weight_cat" ]

model = RandomForestRegressor(max_depth= 5, n_estimators=100)

run_model(feats, model)

(-57.10458753767473, 4.479433414638694)

In [75]:
X = df[feats].values
y = df['prices_amountmin'].values


m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2498 ± 0.0131,brand_cat
0.1058 ± 0.0040,feat_material_cat
0.0239 ± 0.0020,feat_gender_cat
0.0234 ± 0.0017,feat_age_cat
0.0233 ± 0.0006,feat_brand_cat
0.0137 ± 0.0010,feat_weight_cat
0.0043 ± 0.0011,feat_style_cat
0.0017 ± 0.0005,feat_shape_cat
0.0001 ± 0.0000,feat_sport_cat


In [66]:
df["brand"].value_counts(normalize = True)

nike            0.097210
puma            0.033315
ralph lauren    0.028775
vans            0.021116
new balance     0.020295
                  ...   
irish setter    0.000055
blk dnm         0.000055
claiborne       0.000055
gearonic tm     0.000055
x loop          0.000055
Name: brand, Length: 1732, dtype: float64

In [69]:
df[df["brand"] == "nike"].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [77]:

feats = ["brand_cat", "feat_brand_cat", "feat_gender_cat", "feat_material_cat", 
         "feat_style_cat", "feat_age_cat", "feat_shape_cat", "feat_weight_cat", 
         "feat_metal type_cat" ]

model = RandomForestRegressor(max_depth= 5, n_estimators=100)

result = run_model(feats, model)


X = df[feats].values
y = df['prices_amountmin'].values


m = RandomForestRegressor(max_depth= 5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
print(result)
eli5.show_weights(perm, feature_names = feats)

(-57.16917421442791, 4.452697476173263)


Weight,Feature
0.2459 ± 0.0119,brand_cat
0.1047 ± 0.0035,feat_material_cat
0.0268 ± 0.0018,feat_gender_cat
0.0240 ± 0.0009,feat_age_cat
0.0209 ± 0.0007,feat_brand_cat
0.0132 ± 0.0014,feat_weight_cat
0.0088 ± 0.0011,feat_metal type_cat
0.0044 ± 0.0019,feat_style_cat
0.0017 ± 0.0002,feat_shape_cat


In [0]:
!git config --global user.email "smierzchalski.tomek@gmail.com"
!git config --global user.name "tomsmierz"


In [79]:


!git add Dzien5.ipynb



fatal: pathspec 'Dzien5.ipynb' did not match any files


In [80]:
ls


CollabDriveGithub.ipynb  Dzien3.ipynb  dzien4_meta.ipynb  LICENSE
data/                    Dzien4.ipynb  HelloGithub.ipynb  README.md
